# README

- **Author**: `<黃書佑>`
- **Created At**: `<2025-10-14>`
- **Last Modified At**: `<2025-11-29>`

---

## What does this file do?

- `Get DLCs and sequels' release date of each game.`

---

## What does this file take?

- **Source Data Sets**:  
  `Data scraped from Steam` 
  
---

## What does this file output?

- `/data/final/release_events_all_games.csv`  
  - Description: `GameID, Date, Has_DLC, Has_Sequel.`


In [ ]:
# Load packages here
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
# -----------------------------
# 1. 自動篩選條件設定
# -----------------------------
COSMETIC_KEYWORDS = [
    "skin", "pack", "music", "soundtrack", "hairstyle", "outfit",
    "costume", "emote", "voice", "background", "sticker", "wallpaper",
    "accessory", "dlc bundle"
]

# -----------------------------
# 2. 定義函數
# -----------------------------
def get_game_events(app_id: str, sequel_dict=None):
    """
    取得單一遊戲的 DLC + 續作發售事件表。
    sequel_dict: {app_id(str): 'YYYY-MM-DD'}
    """
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    data = requests.get(url).json()

    # 改成用字串索引
    if not data.get(app_id, {}).get("success"):
        return pd.DataFrame()

    game_data = data[app_id]["data"]
    events = []

    # 1. DLC 處理
    if "dlc" in game_data:
        for dlc_id in game_data["dlc"]:
            try:
                dlc_id_str = str(dlc_id)
                dlc_info = requests.get(f"https://store.steampowered.com/api/appdetails?appids={dlc_id_str}", timeout=5).json()
                if not dlc_info.get(dlc_id_str, {}).get("success"):
                    continue
                dlc_data = dlc_info[dlc_id_str]["data"]
                dlc_name = dlc_data.get("name", "").lower()
                release_date = dlc_data.get("release_date", {}).get("date", "")

                # 篩掉外觀型、音樂、皮膚等DLC
                if any(kw in dlc_name for kw in COSMETIC_KEYWORDS):
                    continue

                if release_date:
                    events.append({
                        "GameID": app_id,
                        "Date": pd.to_datetime(release_date, errors="coerce"),
                        "Has_DLC": 1,
                        "Has_Sequel": 0
                    })
            except Exception:
                continue

    # 2. 續作處理
    if sequel_dict and app_id in sequel_dict:
        sequel_dates_list = sequel_dict[app_id]
        # 若只有單一日期（不是list），統一轉成list
        if isinstance(sequel_dates_list, str):
            sequel_dates_list = [sequel_dates_list]
    
        for sequel_date in sequel_dates_list:
            events.append({
                "GameID": app_id,
                "Date": pd.to_datetime(sequel_date),
                "Has_DLC": 0,
                "Has_Sequel": 1
            })

    df = pd.DataFrame(events)
    
    # 安全檢查：如果沒有Date欄位，就直接回傳空表
    if "Date" not in df.columns or df.empty:
        return pd.DataFrame(columns=["GameID", "Date", "Has_DLC", "Has_Sequel"])
    
    df = df.dropna(subset=["Date"]).sort_values("Date").reset_index(drop=True)
    return df



# -----------------------------
# 3. 一次處理多個遊戲
# -----------------------------
def get_all_release_events(game_ids, sequel_dict=None):
    all_events = []
    for gid in tqdm(game_ids, desc="Fetching game events"):
        gid = str(gid)  # 🔹確保為字串
        df = get_game_events(gid, sequel_dict)
        if not df.empty:
            all_events.append(df)
    if not all_events:
        return pd.DataFrame()
    return pd.concat(all_events, ignore_index=True)


# -----------------------------
# 4. 執行
# -----------------------------
# 遊戲 id
game_ids = ['431960', '413150', '381210', '1091500', '367520',                
                '294100', '881100','108600', '3590', '4000',                
                '880940','582660', '323190', '1145360', '244210',                
                '10', '242760', '548430', '457140', '244850',
                '431730', '644930', '814380', '376210', '703080',                
                '588650', '477160', '331790', '233860', '423710'
                ]

# 續作字典的 key
sequel_dates = {
    "367520": ["2025-09-04"],
    "1145360": ["2025-09-25"],
    "244210": ["2019-05-29", "2025-01-16"],
    "10":["2004-03-01", "2004-11-01", "2012-08-21"],
    "242760": ["2024-02-22"],
    "244850": ["2025-01-27"],
    "323190": ["2024-09-20"]
}

# 取得所有遊戲的發售事件表
all_events = get_all_release_events(game_ids, sequel_dict=sequel_dates)

In [ ]:
# -----------------------------
# 5. 儲存結果
# -----------------------------
all_events.to_csv("/Users/sofia/Downloads/release_events_all_games.csv", index=False, encoding="utf-8-sig")